In [1]:
##################################################################
#《Python机器学习及实践：从零开始通往Kaggle竞赛之路（2024年度版）》开源代码
#-----------------------------------------------------------------
#                 @章节号：6.3.3（卷积神经网络的PaddlePaddle实践）                           
#                 @作者：范淼、徐晟桐 
#                 @购书链接：https://item.jd.com/13482761.html
#                 @电子邮箱：fanmiao.cslt.thu@hotmail.com              
#                 @官方交流QQ群号：561500762                        
##################################################################

In [2]:
import paddle
from paddle import nn, optimizer, metric


#设定超参数。
KERNEL_SIZE = 3
FILTERS = (32, 64)
HIDDEN_SIZE = 256
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3


# 搭建卷积神经网络。
paddle_model = nn.Sequential(
    nn.Conv2D(in_channels=1, out_channels=FILTERS[0], kernel_size=KERNEL_SIZE, padding='same'),
    nn.ReLU(),
    
    nn.Conv2D(in_channels=FILTERS[0], out_channels=FILTERS[1], kernel_size=KERNEL_SIZE, padding='same'),
    nn.ReLU(),
    
    nn.MaxPool2D(kernel_size=2, stride=2),
    
    nn.Flatten(),
    
    nn.Linear(in_features=int((28/2) ** 2 * FILTERS[1]), out_features=HIDDEN_SIZE),
    nn.ReLU(),
    
    nn.Linear(in_features=HIDDEN_SIZE, out_features=NUM_CLASSES)
)

#初始化卷积神经网络模型。
model = paddle.Model(paddle_model)

#为模型训练做准备，设置优化器，损失函数和评估指标。
model.prepare(optimizer=optimizer.Adam(learning_rate=LEARNING_RATE, parameters=model.parameters()),
              loss=nn.CrossEntropyLoss(),
              metrics=metric.Accuracy())

In [3]:
import pandas as pd

#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('../datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [4]:
from sklearn.preprocessing import StandardScaler

#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [5]:
from paddle.io import TensorDataset

X_train = X_train.reshape([-1, 1, 28, 28])

X_train = paddle.to_tensor(X_train.astype('float32'))

y_train = y_train.values

#构建适用于PaddlePaddle模型训练的数据集。
train_dataset = TensorDataset([X_train, y_train])

#启动模型训练，指定训练数据集，设置训练轮次，设置每次数据集计算的批次大小。
model.fit(train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 938/938 [==============================] - loss: 0.1193 - acc: 0.8748 - 307ms/step          
Epoch 2/5
step 938/938 [==============================] - loss: 0.1278 - acc: 0.9266 - 2s/step           
Epoch 3/5
step 938/938 [==============================] - loss: 0.0255 - acc: 0.9479 - 879ms/step          
Epoch 4/5
step 938/938 [==============================] - loss: 0.0381 - acc: 0.9651 - 450ms/step          
Epoch 5/5
step 938/938 [==============================] - loss: 0.0320 - acc: 0.9778 - 1s/step          


In [6]:
X_test = X_test.reshape([-1, 1, 28, 28])

X_test = paddle.to_tensor(X_test.astype('float32'))

y_test = y_test.values

#构建适用于PaddlePaddle模型测试的数据集。
test_dataset = TensorDataset([X_test, y_test])

#启动模型测试，指定测试数据集。
result = model.evaluate(test_dataset, verbose=0)

print('卷积神经网络（PaddlePaddle版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %(result['acc'] * 100))

卷积神经网络（PaddlePaddle版本）在fashion_mnist测试集上的准确率为: 91.93%。
